In [1]:
#!pip install kiteconnect

  Obtaining dependency information for kiteconnect from https://files.pythonhosted.org/packages/4c/54/45fb78d0cf6650c74f9ab070147197bd37db8f92e6475e63be1ca837a4ff/kiteconnect-5.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for autobahn[twisted]==19.11.2 from https://files.pythonhosted.org/packages/84/05/828b0c5a558b67c8afd64e233976553298688eee39c78de3a8c8cf98c91f/autobahn-19.11.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for txaio>=18.8.1 from https://files.pythonhosted.org/packages/7d/6c/a53cc9a97c2da76d9cd83c03f377468599a28f2d4ad9fc71c3b99640e71e/txaio-23.1.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/771.5 kB ? eta -:--:--
   ------ --------------------------------- 122.9/771.5 kB 7.0 MB/s eta 0:00:01
   ------------------------- -------------- 491.5/771.5 kB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 771.5/771.5 kB 6.1 MB/s eta 0:00:00


In [1]:
from kiteconnect import KiteConnect
import pandas as pd
api_key = 'yq05y14mp8cidtc2'
client_id = 'CCC807'

In [2]:
kite = KiteConnect(api_key=api_key)

In [3]:
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=yq05y14mp8cidtc2&v=3'

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from requests.exceptions import ConnectionError, Timeout, RequestException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import datetime
import logging

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
driver = webdriver.Chrome(options=chrome_options)
driver.get(kite.login_url())

In [6]:
url = driver.current_url

In [7]:
import re
pattern = r"request_token=([^&]+)"

match = re.search(pattern, url)
if match:
    request_token = match.group(1)
    print("Request token:", request_token)
else:
    print("Request token not found.")

Request token: 8f76lEN3e5rzhbUnTjl6PsltlE0g8LSz


In [8]:
request_token

'8f76lEN3e5rzhbUnTjl6PsltlE0g8LSz'

In [9]:
user = kite.generate_session(request_token, 'uvv5836wd4hxx7efiqvlr4qf33stvqv1')

In [10]:
kite.set_access_token(user["access_token"])

In [24]:
kite.instruments()

[{'instrument_token': 220513285,
  'exchange_token': '861380',
  'tradingsymbol': 'BANKEX25APRFUT',
  'name': 'BANKEX',
  'last_price': 0.0,
  'expiry': datetime.date(2025, 4, 29),
  'strike': 0.0,
  'tick_size': 0.05,
  'lot_size': 30,
  'instrument_type': 'FUT',
  'segment': 'BFO-FUT',
  'exchange': 'BFO'},
 {'instrument_token': 223969029,
  'exchange_token': '874879',
  'tradingsymbol': 'BANKEX25MAYFUT',
  'name': 'BANKEX',
  'last_price': 0.0,
  'expiry': datetime.date(2025, 5, 27),
  'strike': 0.0,
  'tick_size': 0.05,
  'lot_size': 30,
  'instrument_type': 'FUT',
  'segment': 'BFO-FUT',
  'exchange': 'BFO'},
 {'instrument_token': 281854981,
  'exchange_token': '1100996',
  'tradingsymbol': 'BANKEX25JUNFUT',
  'name': 'BANKEX',
  'last_price': 0.0,
  'expiry': datetime.date(2025, 6, 24),
  'strike': 0.0,
  'tick_size': 0.05,
  'lot_size': 30,
  'instrument_type': 'FUT',
  'segment': 'BFO-FUT',
  'exchange': 'BFO'},
 {'instrument_token': 223583237,
  'exchange_token': '873372',
  '

### Getting instrument token

In [50]:
df = pd.read_csv("https://api.kite.trade/instruments", parse_dates=True)
df['expiry'] = pd.to_datetime(df['expiry']).dt.date

In [7]:
import pickle
with open("options_json_for_ui_testing.pkl", "rb") as file:
    symbols_json = pickle.load(file)

In [20]:
symbols_json['DRREDDY']['CE'].keys()

dict_keys([datetime.date(2025, 5, 29)])

In [14]:
token_list = [k for k in symbols_json.keys()]
token_list[0]

'DRREDDY'

In [49]:
pd.to_datetime(df['expiry']).dt.date[0] 

datetime.date(2025, 4, 29)

In [60]:
df[(df['name']==token_list[0]) & (df['instrument_type']=='CE') & (df['expiry']==datetime.date(2025, 5, 29))].iloc[0]['tradingsymbol']

'DRREDDY25MAY1150CE'

### Placing an order

In [3]:
instrument_df = pd.read_csv("https://api.kite.trade/instruments", parse_dates=True)

# Convert the 'expiry' column to datetime.date objects.
instrument_df['expiry'] = pd.to_datetime(instrument_df['expiry']).dt.date

In [4]:
instrument_df[instrument_df['tradingsymbol']=='MON100']

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
8359,136546564,533385,MON100,MOTILAL OSWAL MOST SHARES NASD,0,NaT,0.0,0.01,1,EQ,BSE,BSE
89390,5821185,22739,MON100,MOTILAL OS NASDAQ100 ETF,0,NaT,0.0,0.01,1,EQ,NSE,NSE


In [23]:
try:
    order_id = kite.place_order(tradingsymbol="SETFNIFBK",
                                exchange='NSE',
                                transaction_type='BUY',
                                quantity=1,
                                variety='regular',
                                order_type='MARKET',
                                product='CNC',
                                validity='DAY')

    logging.info("Order placed. ID is: {}".format(order_id))
except Exception as e:
    logging.info("Order placement failed: {}".format(e.message))

In [24]:
order_id

'1909128905567002624'

In [25]:
kite.orders()

[{'account_id': 'CCC807',
  'placed_by': 'CCC807',
  'terminal_id': '111111111111100',
  'order_id': '1909128905567002624',
  'exchange_order_id': '1300000051291992',
  'parent_order_id': None,
  'status': 'COMPLETE',
  'status_message': None,
  'status_message_raw': None,
  'order_timestamp': datetime.datetime(2025, 4, 7, 11, 49, 35),
  'exchange_update_timestamp': '2025-04-07 11:49:35',
  'exchange_timestamp': datetime.datetime(2025, 4, 7, 11, 49, 35),
  'variety': 'regular',
  'modified': False,
  'exchange': 'NSE',
  'tradingsymbol': 'SETFNIFBK',
  'instrument_token': 1884417,
  'order_type': 'MARKET',
  'transaction_type': 'BUY',
  'validity': 'DAY',
  'validity_ttl': 0,
  'product': 'CNC',
  'quantity': 1,
  'disclosed_quantity': 0,
  'price': 0,
  'trigger_price': 0,
  'average_price': 505.99,
  'filled_quantity': 1,
  'pending_quantity': 0,
  'cancelled_quantity': 0,
  'market_protection': 0,
  'meta': {},
  'meta_raw': '{"tag":"","tags":null,"app_id":140050,"gttp":null,"iceber

### Testing zerodha.py file

In [1]:
from zerodha import Zerodha_Api
import pandas as pd

In [2]:
init_zerodha = Zerodha_Api(api_key='yq05y14mp8cidtc2' ,api_secret_key='uvv5836wd4hxx7efiqvlr4qf33stvqv1', client_id='CCC807')

2025-04-08 11:42:44,150 - INFO - Request token extracted: 6lfmgsOGm4G0YqC2v0wzaSbPXRKDhfSE
2025-04-08 11:42:48,105 - INFO - Session generated and access token set.


In [5]:
trading_symbol = Zerodha_Api.extract_instrument_token(name='MOTILAL OS NASDAQ100 ETF', instrument_type='EQ')

In [6]:
trading_symbol

'MON100'

In [7]:
order_id = init_zerodha.place_orders(trading_symbol=trading_symbol, exchange='NSE', transaction_type='BUY',
                                    quantity=3, variety='regular', order_type='MARKET', product='CNC')

2025-04-08 11:48:43,347 - INFO - Order placed. ID is: 1909491074020171776


In [8]:
order_id

'1909491074020171776'

In [9]:
init_zerodha.order_history()

2025-04-08 11:49:08,091 - INFO - Order history retrieved successfully.


[{'account_id': 'CCC807',
  'placed_by': 'CCC807',
  'terminal_id': '111111111111100',
  'order_id': '1909491074020171776',
  'exchange_order_id': '1200000043979573',
  'parent_order_id': None,
  'status': 'COMPLETE',
  'status_message': None,
  'status_message_raw': None,
  'order_timestamp': datetime.datetime(2025, 4, 8, 11, 48, 43),
  'exchange_update_timestamp': '2025-04-08 11:48:43',
  'exchange_timestamp': datetime.datetime(2025, 4, 8, 11, 48, 43),
  'variety': 'regular',
  'modified': False,
  'exchange': 'NSE',
  'tradingsymbol': 'MON100',
  'instrument_token': 5821185,
  'order_type': 'MARKET',
  'transaction_type': 'BUY',
  'validity': 'DAY',
  'validity_ttl': 0,
  'product': 'CNC',
  'quantity': 3,
  'disclosed_quantity': 0,
  'price': 0,
  'trigger_price': 0,
  'average_price': 166.88,
  'filled_quantity': 3,
  'pending_quantity': 0,
  'cancelled_quantity': 0,
  'market_protection': 0,
  'meta': {},
  'meta_raw': '{"tag":"","tags":null,"app_id":140050,"gttp":null,"iceberg":